# Example Usage

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import rasterio as rio
from pathlib import Path
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from src import *


plt.rcParams["figure.figsize"] = (20,10)
GPU_IDX = 0

In [ ]:
import imageio

mwir_img = imageio.imread('imgs/mwir_example.png')[:256,:256,0]/255 # Thermal Infrared Imagery - © [2022] Satellite Vu
rgb_img = imageio.imread('imgs/rgb_example.png')[...,:3]/255

## Cloud Types: Thick Cloud

In [ ]:
cl, mask = add_cloud(rgb_img,
                     return_cloud=True
                    )

plt.subplot(1,3,1)
plt.imshow(rgb_img)
plt.title('Input')
plt.subplot(1,3,2)
plt.imshow(cl)
plt.title('Simulated')
plt.subplot(1,3,3)
plt.imshow(mask)
plt.title('Channel-wise Cloud Mask')

plt.savefig('imgs/thick_cloud.png',
        dpi=200, 
        bbox_inches='tight',
        pad_inches=0.0
       )

## Cloud Types: Thick Foggy Cloud

In [ ]:
cl, mask = add_cloud(rgb_img,
                     min_lvl=0.1,
                     max_lvl=0.5,
                     decay_factor=1.85,
                     return_cloud=True)

plt.subplot(1,3,1)
plt.imshow(rgb_img)
plt.title('Input')
plt.subplot(1,3,2)
plt.imshow(cl)
plt.title('Simulated')
plt.subplot(1,3,3)
plt.imshow(mask)
plt.title('Channel-wise Cloud Mask')

plt.savefig('imgs/foggy_cloud.png',
        dpi=200, 
        bbox_inches='tight',
        pad_inches=0.0
       )

## Cloud Types: Thin Fog

In [ ]:
cl, mask = add_cloud(rgb_img,
                     min_lvl=0.4,
                     max_lvl=0.6,
                     decay_factor=1.5,
                     return_cloud=True)

plt.subplot(1,3,1)
plt.imshow(rgb_img)
plt.title('Input')
plt.subplot(1,3,2)
plt.imshow(cl)
plt.title('Simulated')
plt.subplot(1,3,3)
plt.imshow(mask)
plt.title('Channel-wise Cloud Mask')

plt.savefig('imgs/thin_fog.png',
        dpi=200, 
        bbox_inches='tight',
        pad_inches=0.0
       )

## Thermal Image Example

In [ ]:
cl, mask = add_cloud(mwir_img,
                     max_lvl = 0.6,                    
                     invert=True,
                     blur_scaling=1.0,
                     return_cloud=True
                    )
plt.subplot(1,3,1)
plt.imshow(mwir_img, cmap='gray')
plt.title('Input')
plt.subplot(1,3,2)
plt.imshow(cl,vmin=0.0,vmax=1.0, cmap='gray')
plt.title('Simulated')
plt.subplot(1,3,3)
plt.imshow(mask,vmin=0.0,vmax=1.0, cmap='gray')
plt.title('Channel-wise Cloud Mask')

plt.savefig('imgs/mwir_cloud.png',
        dpi=200, 
        bbox_inches='tight',
        pad_inches=0.0
       )

## Gradual Cloud Increase
An interesting result can be obtained by repeatedly applying the cloud augmentation. Eventually, the image becomes dominated by the cloud component.

In [ ]:
new = rgb_img

plt.figure(figsize=(10,20))
for step in range(32):
    
    plt.subplot(8,4,1+step)
    plt.imshow(new.clip(0,1))
    
    lims = sorted([0.6+0.4*el for el in np.random.rand(2).tolist()])
    
    new = add_cloud(new,
                    min_lvl=0.5,
                    max_lvl=1.0)

# Parameter Ablations

## 1. Basic Cloud
A basic cloud with `max_lvl` of ground component (`1.0` guarantees *some* clear pixels) and `min_lvl` that controles the maximum value of cloud strength (`1.0` guarantees *some* pixels with non-transparent cloud).

In [ ]:
cl, mask = add_cloud(rgb_img,
                     min_lvl=0.0,
                     max_lvl=1.0,
                     cloud_color=False,
                     channel_offset=0,
                     blur_scaling=0,
                     return_cloud=True)

plt.subplot(1,3,1)
plt.imshow(rgb_img)
plt.title('Input')
plt.subplot(1,3,2)
plt.imshow(cl)
plt.title('Simulated')
plt.subplot(1,3,3)
plt.imshow(mask)
plt.title('Channel-wise Cloud Mask')

plt.savefig('imgs/basic_cloud.png',
        dpi=200, 
        bbox_inches='tight',
        pad_inches=0.0
       )

## 2. Cloud Color
The color of the cloud component (white by default) can be adjusted with the average color from the clear underlying image by setting `cloud_color` to `True`.

In [ ]:
cl, mask = add_cloud(rgb_img,
                     min_lvl=0.0,
                     max_lvl=1.0,
                     cloud_color=True,
                     return_cloud=True
                    )

plt.subplot(1,3,1)
plt.imshow(rgb_img)
plt.title('Input')
plt.subplot(1,3,2)
plt.imshow(cl)
plt.title('Simulated')
plt.subplot(1,3,3)
plt.imshow(mask)
plt.title('Channel-wise Cloud Mask')

plt.savefig('imgs/cloud_color.png',
        dpi=200, 
        bbox_inches='tight',
        pad_inches=0.0
       )

## 3. Channel Offset
The `channel_offset` setting controls the maximum positional offset in `x` or `y` direction between the channel-specific cloud masks and the base cloud mask (The value is sampled uniformly from the range `[-channel_offset,+channel_offset]`).

In [ ]:
cl, mask = add_cloud(rgb_img,
                     min_lvl=0.0,
                     max_lvl=1.0,
                     channel_offset=3,
                     return_cloud=True
                    )

plt.subplot(1,3,1)
plt.imshow(rgb_img)
plt.title('Input')
plt.subplot(1,3,2)
plt.imshow(cl)
plt.title('Simulated')
plt.subplot(1,3,3)
plt.imshow(mask)
plt.title('Channel-wise Cloud Mask')

plt.savefig('imgs/channel_offset.png',
        dpi=200, 
        bbox_inches='tight',
        pad_inches=0.0
       )

## 4. Blur Under Cloud
By setting `blur_scaling` argument to a value different than `0.0`, a locally varying Gaussian blur can be applied. The standard deviation is proportional to the cloud strength (ranging between `0` and `1`) multplied by the `blur_scaling` factor.

In [ ]:
cl, mask = add_cloud(rgb_img,
                     min_lvl=0.3,
                     max_lvl=1.0,
                     blur_scaling=4.0,
                     return_cloud=True
                    )

plt.subplot(1,3,1)
plt.imshow(rgb_img)
plt.title('Input')
plt.subplot(1,3,2)
plt.imshow(cl)
plt.title('Simulated')
plt.subplot(1,3,3)
plt.imshow(mask)
plt.title('Channel-wise Cloud Mask')

plt.savefig('imgs/back_blur.png',
        dpi=200, 
        bbox_inches='tight',
        pad_inches=0.0
       )